# Looking back 2019 through Hacker News and BigQuery
date: 2020-01-01T07:12:54-08:00
<!--eofm-->

This is a re-do of [Looking back 2017, through Hacker News](https://blog.8-p.info/en/2018/01/01/hacker-news-2017/). Google Cloud Platform is hosting some public datasets which are readily accessible from BigQuery. In this article, I'm going to check what was happening in 2019, through the stories on Hacker News.

### Setup

Previously I was using `%bq` magic, but it seems deprecated and now `%bigquery` is the way, according to [Migrating from the datalab Python package](https://cloud.google.com/bigquery/docs/datalab-migration).

In [1]:
# To use "%%bigquery"
%load_ext google.cloud.bigquery

For the BigQuery client library, I need to [setup authentication by setting GOOGLE_APPLICATION_CREDENTIALS environment variable](https://cloud.google.com/bigquery/docs/reference/libraries#setting_up_authentication). Without that, you would get `Project was not passed and could not be determined from the environment`.

The dictionary below is used to parameterize queries below. Updating this notebook for 2011 would be just changing the cell below.

In [2]:
params = {"year": 2019}

Then import pandas for just setting max_colwidth. Without that, pandas turncates long strings, such as URLs

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

### Most Popular (Upvoted) Stories
[Google Cloud Platform's dataset](https://console.cloud.google.com/marketplace/details/y-combinator/hacker-news?pli=1) doesn't explain the schema, but mentioned that the dataset is based on [Hacker News API](https://github.com/HackerNews/API).

In [4]:
%%bigquery --params $params
SELECT title, timestamp, `by`, score, descendants as comments, url, id FROM `bigquery-public-data.hacker_news.full`
WHERE type = 'story' AND extract(year FROM timestamp) = @year
ORDER BY score DESC LIMIT 50

,title,timestamp,by,score,comments,url,id
0,Switch from Chrome to Firefox,2019-05-30 16:09:19+00:00,WisNorCan,3287,981,https://www.mozilla.org/en-US/firefox/switch/,20052623
1,I Sell Onions on the Internet,2019-04-23 13:00:24+00:00,eightturn,3015,435,https://www.deepsouthventures.com/i-sell-onions-on-the-internet/,19728132
2,Announcing unlimited free private repos,2019-01-07 17:03:59+00:00,razer6,2867,684,https://blog.github.com/2019-01-07-new-year-new-github/,18847043
3,Slack’s new WYSIWYG input box is terrible,2019-11-20 23:13:09+00:00,ingve,2776,1076,https://quuxplusone.github.io/blog/2019/11/20/slack-rich-text-box/,21589647
4,Show HN: A retro video game console I've been working on in my free time,2019-03-14 20:25:03+00:00,pkiller,2690,210,https://internalregister.github.io/2019/03/14/Homebrew-Console.html,19393279
5,My Business Card Runs Linux,2019-12-24 10:15:42+00:00,rcarmo,2584,397,https://www.thirtythreeforty.net/posts/2019/12/my-business-card-runs-linux/,21871026
6,Blizzard Suspends Professional Hearthstone Player for Hong Kong Comments,2019-10-08 09:23:08+00:00,hownottowrite,2525,1126,https://playhearthstone.com/en-us/blog/23179289/,21190265
7,Raspberry Pi 4,2019-06-24 06:00:28+00:00,MarcScott,2504,837,https://www.raspberrypi.org/blog/raspberry-pi-4-on-sale-now-from-35,20260863
8,Twitter to ban political advertising,2019-10-30 20:07:19+00:00,coloneltcb,2447,1004,https://twitter.com/jack/status/1189634360472829952,21401973
9,"No Thank You, Mr. Pecker",2019-02-07 22:52:16+00:00,coloneltcb,2444,730,https://medium.com/@jeffreypbezos/no-thank-you-mr-pecker-146e3922310f,19109474


### Commonly Shared/Upvoted Domains

Note that `NO DOMAIN` means stories without associated URLs, such as "Ask HN".

In [5]:
%%bigquery --params $params
SELECT
    domains_this_year.domain,

    domains_this_year.count as count_this_year,
    domains_last_year.count as count_last_year,
    (domains_this_year.count / domains_last_year.count) as count_yoy,

    domains_this_year.score as score_this_year,
    domains_last_year.score as score_last_year,
    (domains_this_year.score / domains_last_year.score) as score_yoy
FROM
    (SELECT
     domain, COUNT(1) AS count, SUM(score) AS score FROM
        (SELECT
         IFNULL(REGEXP_EXTRACT(url,r'^https?://(?:www.)?([^/]*)/?(?:.*)'), "NO DOMAIN") AS domain, score
         FROM `bigquery-public-data.hacker_news.full`
         WHERE extract(year FROM timestamp) = @year)
     GROUP BY domain
     ORDER BY count DESC LIMIT 100) domains_this_year
    JOIN
    (SELECT
     domain, COUNT(1) AS count, SUM(score) AS score FROM
         (SELECT
          IFNULL(REGEXP_EXTRACT(url,r'^https?://(?:www.)?([^/]*)/?(?:.*)'), "NO DOMAIN")  AS domain, score
          FROM `bigquery-public-data.hacker_news.full`
          WHERE extract(year FROM timestamp) = (@year-1))
     GROUP BY domain
     ORDER BY count DESC LIMIT 100) domains_last_year
    ON domains_this_year.domain = domains_last_year.domain
LIMIT 50

,domain,count_this_year,count_last_year,count_yoy,score_this_year,score_last_year,score_yoy
0,NO DOMAIN,2786600,2420204,1.151391,229058,241363,0.949019
1,medium.com,17424,18113,0.961961,103059,117330,0.878369
2,github.com,13539,13336,1.015222,262248,228399,1.148201
3,youtube.com,8008,7972,1.004516,47554,48127,0.988094
4,nytimes.com,6643,5769,1.151499,174981,159671,1.095885
5,en.wikipedia.org,4566,2779,1.643037,52026,28513,1.824641
6,theguardian.com,3926,3481,1.127837,59598,77924,0.764822
7,bloomberg.com,3863,3469,1.113577,103875,123642,0.840127
8,twitter.com,3775,2460,1.534553,102478,48913,2.095108
9,arstechnica.com,3231,3554,0.909116,44255,56575,0.782236


### See Also
- [Looking back at 9 years of Hacker News](http://debarghyadas.com/writes/looking-back-at-9-years-of-hacker-news/) (2015) -- More detailed analysis of Hacker News from 2006 to 2015. A lof of queries on this article are based on the analysis.
- [Hacker News stats by domain TLD](http://blog.park.io/articles/hacker-news-posts-by-domain-tld/) (2015)